In [2]:
from gerrychain import Graph, Election, updaters, Partition, MarkovChain, constraints
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from gerrychain.tree import random_spanning_tree, PopulatedGraph, predecessors
from collections import deque, namedtuple
from functools import partial
from collections import Counter
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
Cut = namedtuple("Cut", "edge subset")


##########################################################################################
#     THIS IS NO LONGER NEEDED, BUT IS HANDY FOR CHECKING THE SIZE OF THE PARTITIONS     #
#                 TURN IT OFF AND REMOVE CONSTRAINT FOR ACTUAL RUNS                      #
##########################################################################################
# def stop_disappearing(partition):                                                      #
#     if not partition.parent:                                                           #
#         return True                                                                    #
#     sizes_now = [len(part) for part in partition.assignment.parts.values()]            #
#     sizes_past = [len(part) for part in partition.parent.assignment.parts.values()]    #
#     print(sizes_now, min(sizes_past), max(sizes_past))                                 #
#     return all(len(part) > 1 for part in partition.assignment.parts.values())          #
##########################################################################################


def find_balanced_edge_cuts(h, choice=random.choice):
    root = choice([x for x in h if h.degree(x) > 2])
    pred = predecessors(h.graph, root)
    cuts = []
    leaves = deque(x for x in h if h.degree(x) == 1)
    while len(leaves) > 0:
        leaf = leaves.popleft()
        cuts.append(Cut(edge=(leaf, pred[leaf]), subset=h.subsets[leaf].copy()))
        parent = pred[leaf]
        h.contract_node(leaf, parent)
        if h.degree(parent) == 1 and parent != root:
            leaves.append(parent)
            
    even_split = int(len(h.graph.nodes)/2)
    #cut_lengths is number of nodes in one of the districts after cut of a specific edge
    cut_lengths = np.asarray([len(cuts[i].subset) for i in range(len(cuts))])
    cut_distances = np.abs(cut_lengths-even_split)
    cut_distances, cuts = zip(*sorted(zip(cut_distances, cuts)))
    cut_distances = list(filter(lambda a: a != max(cut_distances), cut_distances))
    
    temp_d = {}
    for i in range(len(cut_distances)):
        temp_d[i] = cut_distances[i]
    #print(temp_d)
        # indices of cut_distances
    population = list(range(len(cut_distances)))
    weight_small_d = (1/(np.asarray(cut_distances)**2+1))
    probs = weight_small_d/sum(weight_small_d)
#     print(probs)
    choice = random.choices(population, probs)[0]
#     print(choice)
############################################################################## 
#   VERIFIES THAT YOU ARE SAMPLING FROM ALL POSSIBLE EDGE CHOICES CORRECTLY  #
##############################################################################
#     d = Counter(random.choices(population, probs, k=100000))               #
#     fig = plt.figure(figsize=(12,12))                                      #
#     ax = fig.add_subplot(2, 1, 1)                                          #
#     plt.bar(d.keys(), d.values(), width=1.0, color='g')                    #
#     ax.set_yscale('log')                                                   #
#     plt.show()                                                             #
#     print(d)                                                               #
##############################################################################
    return [cuts[choice]]


def partition_tree(graph, pop_col, pop_target, epsilon, node_repeats=1, spanning_tree=None, choice=random.choice):
    populations = {node: graph.nodes[node][pop_col] for node in graph}
    possible_cuts = []
    spanning_tree = random_spanning_tree(graph)
    while len(possible_cuts) == 0:
        spanning_tree = random_spanning_tree(graph)
        h = PopulatedGraph(spanning_tree, populations, pop_target, epsilon)
        possible_cuts = find_balanced_edge_cuts(h, choice=choice)
    return choice(possible_cuts).subset

In [3]:
###################################################################
#                    *HASHING AND COMPRESSING*                    #
# THE PythonObjectEncoder CLASS SERIALIZES GENERIC PYTHON OBJECTS #
#         THESE OBJECTS CAN THEN BE COMPRESSED AND SAVED          #
###################################################################
from json import JSONEncoder, dumps, loads
from io import BytesIO
import hashlib
import pickle
import gzip
from os import path


class PythonObjectEncoder(JSONEncoder):
    def __init__(self, partition):
        self.partition = partition
        self.keys = sorted(list(initial_partition.parts.keys()))
        self.objstrings = []
        self.hashstrings = []
        self.d_plan = dict.fromkeys(self.keys)
        
        
    def __call__(self, obj):
        if isinstance(obj, (list, dict, str, bytes, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}
    
    
    def update(self, partition):
        for i in range(len(self.keys)):
            self.d_plan[self.keys[i]] = list(partition.assignment.parts[self.keys[i]])
        encoded_plan = dumps(self.d_plan)
        compressed_plan = PythonObjectEncoder.compressStringToBytes(encoded_plan)
        hashstring = hashlib.md5(encoded_plan.encode('utf-8')).hexdigest()
        self.objstrings.append(compressed_plan)
        self.hashstrings.append(hashstring)
        
        
    def commit(self, file_name):
        final_table = dict(zip(self.hashstrings,self.objstrings))
        self.save_obj(final_table, file_name)
        
        
    def load_maps(self, file_name):
        decode_test = self.load_obj(file_name)
        recreated_maps = []
        for key in decode_test:
            recreated_maps.append(loads(PythonObjectEncoder.decompressBytesToString(decode_test[key])))
        return recreated_maps
        
    
    @staticmethod
    def compressStringToBytes(inputString):
        bio = BytesIO()
        bio.write(inputString.encode("utf-8"))
        bio.seek(0)
        stream = BytesIO()
        compressor = gzip.GzipFile(fileobj=stream, mode='w')
        while True:
            chunk = bio.read(8192)
            if not chunk:
                compressor.close()
                return stream.getvalue()
            compressor.write(chunk)

            
    @staticmethod
    def decompressBytesToString(inputBytes):
        bio = BytesIO()
        stream = BytesIO(inputBytes)
        decompressor = gzip.GzipFile(fileobj=stream, mode='r')
        while True:
            chunk = decompressor.read(8192)
            if not chunk:
                decompressor.close()
                bio.seek(0)
                return bio.read().decode("utf-8")
            bio.write(chunk)
        return None
    
    
    @staticmethod
    def save_obj(obj, name):
        if path.exists(name + '.pkl'):
            prev_map = PythonObjectEncoder.load_obj(name)
            final_map = {**prev_map, **obj}
            with open(name + '.pkl', 'rb+') as f:
                pickle.dump(final_map, f, pickle.HIGHEST_PROTOCOL)
        else:
            with open(name + '.pkl', 'wb') as f:
                pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
        
    @staticmethod
    def load_obj(name):
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)

In [5]:
filename = "TX_vtds_fixed"
graph = Graph.from_json(f"{filename}.json")

office, year = "PRES", "16"
pop_used = "TOTPOP"
area = "area"
assign = "USCD"

elect = office + year
elect_D = elect + "D"
elect_R = elect + "R"

elections = Election(elect, {"Dem": elect_D, "Rep": elect_R})

my_updaters = {"population": updaters.Tally(pop_used, alias="population"), elect: elections}
initial_partition = Partition(
    graph,
    assignment=assign,
    updaters=my_updaters
)

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(
    recom,
    pop_col=pop_used,
    pop_target=ideal_population,
    epsilon=1,
    node_repeats=2,
    method=partition_tree
)

chain = MarkovChain(
    proposal=proposal,
    accept=always_accept,
    constraints = [constraints.contiguous],
    initial_state=initial_partition,
    total_steps=5
)

In [6]:
%%time

poe = PythonObjectEncoder(initial_partition)

for partition in chain:
    poe.update(partition)
    print("running")

running
running
running
running
running
CPU times: user 1.04 s, sys: 0 ns, total: 1.04 s
Wall time: 1.04 s


In [7]:
%%time
poe.commit('test')

CPU times: user 2.39 ms, sys: 0 ns, total: 2.39 ms
Wall time: 1.18 ms


In [8]:
%%time
reloaded_maps = poe.load_maps('test')

CPU times: user 14.2 ms, sys: 0 ns, total: 14.2 ms
Wall time: 11.7 ms
